In [ ]:
import tensorflow as tf
from keras.optimizers import Adam
import pandas as pd
import numpy as np
from transformers import TFBartForConditionalGeneration, AutoTokenizer, BartTokenizerFast

In [ ]:
# Original BART model
tokenizer = BartTokenizerFast.from_pretrained("facebook/bart-base")
model = TFBartForConditionalGeneration.from_pretrained("facebook/bart-base")

# Testing the base model
input_text = "The tower is 324 metres (1,063 ft) tall, about the same height as an 81-storey building, and the tallest structure in Paris. Its base is square, measuring 125 metres (410 ft) on each side. During its construction, the Eiffel Tower surpassed the Washington Monument to become the tallest man-made structure in the world, a title it held for 41 years until the Chrysler Building in New York City was finished in 1930. It was the first structure to reach a height of 300 metres. Due to the addition of a broadcasting aerial at the top of the tower in 1957, it is now taller than the Chrysler Building by 5.2 metres (17 ft). Excluding transmitters, the Eiffel Tower is the second tallest free-standing structure in France after the Millau Viaduct."

inputs = tokenizer(input_text, return_tensors="tf", max_length=2**20, truncation=True)
summary_ids = model.generate(inputs["input_ids"])

print(tokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False))

In [3]:
# Load in data for fine tuning
wiki_train = pd.read_csv("./datasets/pwkp.csv")

wiki_tokens_in = tokenizer(wiki_train["Original Data"].tolist(), padding=True, return_tensors='np')
wiki_tokens_out = tokenizer(wiki_train["Highlights Data"].tolist(), padding=True, return_tensors='np')
wiki_in = dict(wiki_tokens_in)
wiki_out = dict(wiki_tokens_out)

In [ ]:
model.compile(optimizer=Adam(3e-5))
model.fit(wiki_in["input_ids"], wiki_out["input_ids"], epochs=1)

In [ ]:
model.save("./", save_format='tf')